<a href="https://colab.research.google.com/github/rohandawar/pyspark/blob/main/RandomForestRegressor.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In this notebook, I am trying a Rabdom forest Regressor

In [2]:
!pip install pyspark

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 310.8/310.8 MB 4.2 MB/s eta 0:00:00
  Preparing metadata (setup.py) ... done
  Created wheel for pyspark: filename=pyspark-3.4.1-py2.py3-none-any.whl size=311285398 sha256=a5bca6de852f885d002929c03a221c6ca9b34739d4baf9d27f30edcf6119757c
  Stored in directory: /root/.cache/pip/wheels/0d/77/a3/ff2f74cc9ab41f8f594dabf0579c2a7c6de920d584206e0834
Successfully built pyspark


In [3]:
# Import Libs

#Pyspark
from pyspark.sql import SparkSession
from pyspark.ml.feature import VectorAssembler
from pyspark.ml.regression import RandomForestRegressor
from pyspark.ml.evaluation import RegressionEvaluator

# Google Colab
from google.colab import drive



In [4]:
# load the drive
drive.mount('/content/drive')

Mounted at /content/drive


In [5]:
# Start the spark session
spark = SparkSession.builder.appName('RandomForestRegressor').getOrCreate()

In [6]:
!ls /content/drive/MyDrive/DataSets_Pyspark_GoogleColab_Primer/Boston.csv

/content/drive/MyDrive/DataSets_Pyspark_GoogleColab_Primer/Boston.csv


In [7]:
df = spark.read.csv('/content/drive/MyDrive/DataSets_Pyspark_GoogleColab_Primer/Boston.csv', inferSchema=True, header=True).drop('_c0')
df.show(5)

+-------+----+-----+----+-----+-----+----+------+---+---+-------+------+-----+----+
|   crim|  zn|indus|chas|  nox|   rm| age|   dis|rad|tax|ptratio| black|lstat|medv|
+-------+----+-----+----+-----+-----+----+------+---+---+-------+------+-----+----+
|0.00632|18.0| 2.31|   0|0.538|6.575|65.2|  4.09|  1|296|   15.3| 396.9| 4.98|24.0|
|0.02731| 0.0| 7.07|   0|0.469|6.421|78.9|4.9671|  2|242|   17.8| 396.9| 9.14|21.6|
|0.02729| 0.0| 7.07|   0|0.469|7.185|61.1|4.9671|  2|242|   17.8|392.83| 4.03|34.7|
|0.03237| 0.0| 2.18|   0|0.458|6.998|45.8|6.0622|  3|222|   18.7|394.63| 2.94|33.4|
|0.06905| 0.0| 2.18|   0|0.458|7.147|54.2|6.0622|  3|222|   18.7| 396.9| 5.33|36.2|
+-------+----+-----+----+-----+-----+----+------+---+---+-------+------+-----+----+
only showing top 5 rows



In [13]:
# getting the predictor Columns
col_list = df.columns

col_list.remove('medv')

# instiate a vector Assembler
vec_assembler = VectorAssembler(inputCols=col_list, outputCol ='predictors' )

vector_output = vec_assembler.transform(df)


# Diplay
vector_output.show(5)


+-------+----+-----+----+-----+-----+----+------+---+---+-------+------+-----+----+--------------------+
|   crim|  zn|indus|chas|  nox|   rm| age|   dis|rad|tax|ptratio| black|lstat|medv|          predictors|
+-------+----+-----+----+-----+-----+----+------+---+---+-------+------+-----+----+--------------------+
|0.00632|18.0| 2.31|   0|0.538|6.575|65.2|  4.09|  1|296|   15.3| 396.9| 4.98|24.0|[0.00632,18.0,2.3...|
|0.02731| 0.0| 7.07|   0|0.469|6.421|78.9|4.9671|  2|242|   17.8| 396.9| 9.14|21.6|[0.02731,0.0,7.07...|
|0.02729| 0.0| 7.07|   0|0.469|7.185|61.1|4.9671|  2|242|   17.8|392.83| 4.03|34.7|[0.02729,0.0,7.07...|
|0.03237| 0.0| 2.18|   0|0.458|6.998|45.8|6.0622|  3|222|   18.7|394.63| 2.94|33.4|[0.03237,0.0,2.18...|
|0.06905| 0.0| 2.18|   0|0.458|7.147|54.2|6.0622|  3|222|   18.7| 396.9| 5.33|36.2|[0.06905,0.0,2.18...|
+-------+----+-----+----+-----+-----+----+------+---+---+-------+------+-----+----+--------------------+
only showing top 5 rows



In [14]:
# split the data into training & testing set
train_df, test_df = vector_output.randomSplit([0.7,0.3], seed=42)

In [17]:
# Instatiate a random orest Regressor
regressor = RandomForestRegressor(featuresCol='predictors', labelCol = 'medv')

In [18]:
# Lets Fit the model
rf_model = regressor.fit(train_df)

In [20]:
# Lets the make the predictions
pred_df = rf_model.transform(test_df)

# Display
pred_df.show(5)

+-------+----+-----+----+-----+-----+----+------+---+---+-------+------+-----+----+--------------------+------------------+
|   crim|  zn|indus|chas|  nox|   rm| age|   dis|rad|tax|ptratio| black|lstat|medv|          predictors|        prediction|
+-------+----+-----+----+-----+-----+----+------+---+---+-------+------+-----+----+--------------------+------------------+
|0.01096|55.0| 2.25|   0|0.389|6.453|31.9|7.3073|  1|300|   15.3|394.72| 8.23|22.0|[0.01096,55.0,2.2...|24.370127301988965|
|0.01381|80.0| 0.46|   0|0.422|7.875|32.0|5.6484|  4|255|   14.4|394.23| 2.97|50.0|[0.01381,80.0,0.4...| 43.96691969730476|
|0.01439|60.0| 2.93|   0|0.401|6.604|18.8|6.2196|  1|265|   15.6| 376.7| 4.38|29.1|[0.01439,60.0,2.9...| 28.91998787201959|
|0.01501|80.0| 2.01|   0|0.435|6.635|29.7| 8.344|  4|280|   17.0|390.94| 5.99|24.5|[0.01501,80.0,2.0...| 26.42872920660784|
|0.01778|95.0| 1.47|   0|0.403|7.135|13.9|7.6534|  3|402|   17.0| 384.3| 4.45|32.9|[0.01778,95.0,1.4...| 32.40051717719978|
+-------

In [22]:
# Lets Evaulate the model
regression_evual = RegressionEvaluator(predictionCol='prediction', labelCol='medv', metricName='rmse')

rmse = regression_evual.evaluate(pred_df)
r2 = regression_evual.setMetricName('r2').evaluate(pred_df)
print(f"RMSE:{rmse}")
print(f"R2:{r2}")

RMSE:3.772920133335278
R2:0.8519132595904466
